# Problem: Sagemaker SDK is not compatible with type checkers

This notebook:
- demonstrates the challenges that results from the fact that the Sagemaker SDK does not conform to [PEP 561](https://peps.python.org/pep-0561/): Even though it is type annotated, the type annotations are not used by mypy, because the [Sagemaker SDK is missing a py.typed file that would communicate to the type checker that the library is in fact typed](https://github.com/aws/sagemaker-python-sdk/issues/2985)).
- shows how to resolve this problem.

In [1]:
# Enable type checking
%load_ext nb_mypy

Version 1.0.5


In [2]:
from abc import ABC, abstractmethod
from typing import Generic, TypeVar

from sagemaker.processing import Processor, FrameworkProcessor
from sagemaker.sklearn.estimator import SKLearn

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/thomas-22/.config/sagemaker/config.yaml


For example, let's define an interface for a simple factory for various Processors. We first correctly implement this interface for a FrameworkProcessor.

In [3]:
# Interface
class ProcessorFactoryInterface(ABC):
    @abstractmethod
    def create_processor(self) -> Processor:
        ...

# Implementation
class FrameworkProcessorFactory(ProcessorFactoryInterface):
    def create_processor(self) -> FrameworkProcessor:
        return FrameworkProcessor(
            estimator_cls=SKLearn,
            framework_version="0.23-1",
            role="SageMakerRole",
            instance_count=1,
            instance_type="ml.m5.xlarge",
            max_runtime_in_seconds=1200,
        )

Now let's try what happens if we do not implement this interface incorrectly: We simply make the factory method return an integer, which is clearly a violation of the interface. However, despite this, **no error is raised.** 

Let's first make sure to undo the fix I propose later, in case this notebook is run multiple times>

In [4]:
# Return package to default state
! (rm $(poetry env info --path)/lib/python3.10/site-packages/sagemaker/py.typed &> /dev/null) || echo "No py.typed file found"

Now we implement this interface improperly.

In [5]:
class BuggyProcessor(ProcessorFactoryInterface):
    # This method has the wrong return type
    def create_processor(self) -> int:
        return 1

This is especially insidious if we run mypy from a notebook, because it does not alert us that it basically treats `Processor` as `Any``. (I haven't investigated whether there are some configs we can tweak, but I don't think this is the case, because I'm using the defaults for both nb_mypy and mypy).

To see the difference, let's write this code to a file and analyze it directly with mypy:

In [6]:
%%writefile tmp/buggy_processor.py
from abc import ABC, abstractmethod
from typing import Generic, TypeVar
from sagemaker.processing import Processor, FrameworkProcessor
from sagemaker.sklearn.estimator import SKLearn

class ProcessorFactory(ABC):
    @abstractmethod
    def create_processor(self) -> Processor:
        ...

class FrameworkProcessorFactory(ProcessorFactory):
    def create_processor(self) -> FrameworkProcessor:
        return FrameworkProcessor(
            estimator_cls=SKLearn,
            framework_version="0.23-1",
            role="SageMakerRole",
            instance_count=1,
            instance_type="ml.m5.xlarge",
            max_runtime_in_seconds=1200,
        )

#  ================ This is the new part ======================
class BuggyProcessor(ProcessorFactory):
    def create_processor(self) -> int:
        return 1


Overwriting tmp/buggy_processor.py


In [7]:
! mypy tmp/buggy_processor.py

tmp/buggy_processor.py:3: error: Skipping analyzing "sagemaker.processing": module is installed, but missing library stubs or py.typed marker  [import-untyped]
tmp/buggy_processor.py:4: error: Skipping analyzing "sagemaker.sklearn.estimator": module is installed, but missing library stubs or py.typed marker  [import-untyped]
tmp/buggy_processor.py:4: note: See https://mypy.readthedocs.io/en/stable/running_mypy.html#missing-imports
Found 2 errors in 1 file (checked 1 source file)


We see that mypy does warn us that it is skipping analyzing "sagemaker.processing", though it is still easy to overlook the implications for our interface.

# Solution
I decided to **manually add the py.typed marker** to the local library install, as suggested [here](https://github.com/aws/sagemaker-python-sdk/issues/2985). This is pretty easy, since it is just an empty file.

In [8]:
! touch $(poetry env info --path)/lib/python3.10/site-packages/sagemaker/py.typed

If we define the interface improperly (same example as above), we now do get an error during static analysis:

In [9]:
class BuggyProcessor(ProcessorFactoryInterface):
    def create_processor(self) -> int:
        return 1

<cell>2: error: Return type "int" of "create_processor" incompatible with return type "Processor" in supertype "ProcessorFactoryInterface"  [override]


As expected, if we run mypy directly, we don't get any complaints anymore about missing library stubs/py.typed marker.

In [10]:
! mypy tmp/buggy_processor.py

tmp/buggy_processor.py:24: error: Return type "int" of "create_processor" incompatible with return type "Processor" in supertype "ProcessorFactory"  [override]
Found 1 error in 1 file (checked 1 source file)


Let me stress that **I'm not completely happy with the solution because it is somewhat brittle**, as we have to remember to manually perform this step for new installs or after updating the library. Unfortunately, though, none of the [other solution](https://mypy.readthedocs.io/en/stable/running_mypy.html#missing-library-stubs-or-py-typed-marker) are available, e.g., installing a separate package with stubs ([like there is for boto3](https://pypi.org/project/boto3-stubs/)). 

Thus, **I see this as the best strategy, including for production use** (probably with some additional automation step), **because the alternative is to forego type analysis altogether, which is even more brittle.** Remember that this manual solution does not have any impact at runtime, so we would not do this for the infrastructure running the actual production code. Instead, this modification would only take place on the infrastructure where static analysis is run (development machines, CI/CD infra).